# Lean splitter

In [ ]:
# Cell 1: Create and Build the Extractor
import os

with open("Extract.lean", "w") as f:
    f.write("""
import Lean
open Lean Elab Frontend Meta Json

structure DeclInfo where
  name : String
  type : String
  proofTerm : String
  dependencies : List String
  isProp : Bool
  isAbbrev : Bool  -- NEW: Tracks if this is an 'abbrev'
  deriving ToJson

-- HELPER: Properly check for substrings
def containsSubstr (s substr : String) : Bool :=
  (s.splitOn substr).length > 1

unsafe def main (args : List String) : IO UInt32 := do
  if args.length != 1 then
    IO.println "Usage: lake exe extract <filename.lean>"
    return 1

  let fileName := args.head!
  let input ← IO.FS.readFile fileName

  let sysroot ← Lean.findSysroot
  let leanPath ← IO.getEnv "LEAN_PATH"
  let sp : System.SearchPath := match leanPath with
    | some p => System.SearchPath.parse p
    | none => []
  Lean.initSearchPath sysroot sp

  let inputCtx := Parser.mkInputContext input fileName
  let (header, parserState, messages) ← Parser.parseHeader inputCtx
  let (env, messages) ← processHeader header {} messages inputCtx

  let cmdState := Command.mkState env messages {}
  let frontendState ← IO.processCommands inputCtx parserState cmdState
  let env := frontendState.commandState.env

  let mut decls : List DeclInfo := []
  let fileMap := FileMap.ofString input

  let opts : Options := Options.empty
    |>.insert `pp.maxDepth (DataValue.ofNat 20000)
    |>.insert `pp.width (DataValue.ofNat 1000)
    |>.insert `pp.deepTerms (DataValue.ofBool false)
    |>.insert `pp.explicit (DataValue.ofBool true)   -- CRITICAL: Prints @Nat.le_trans 11 21 ...
    |>.insert `pp.fullNames (DataValue.ofBool true)  -- CRITICAL: Avoids naming ambiguity
    |>.insert `pp.analyze (DataValue.ofBool true)    -- CRITICAL: Adds type annotations

  let coreCtx : Core.Context := { fileName := fileName, fileMap := fileMap, options := opts }
  let coreSt : Core.State := { env := env }

  let constants := env.constants.map₂

  for (name, cinfo) in constants do
    let nameStr := name.toString

    let isInternal := containsSubstr nameStr "match_" ||
                      containsSubstr nameStr "proof_" ||
                      containsSubstr nameStr ".eq_" ||
                      containsSubstr nameStr ".injEq" ||
                      name.isInternal

    if (env.getModuleIdxFor? name).isNone && !isInternal then
      if cinfo.hasValue then
        try
          let (typeStr, valStr, deps, isP) ← (MetaM.run' <| do
              let type ← ppExpr cinfo.type
              let valExpr := cinfo.value?.getD (Expr.sort Level.zero)
              let valPp ← ppExpr valExpr
              let used := valExpr.getUsedConstants

              let validDeps := used.toList.filterMap fun n =>
                let nStr := n.toString
                if !(containsSubstr nStr "match_") && !(containsSubstr nStr "proof_") && !(containsSubstr nStr ".eq_") then
                  some nStr
                else
                  none

              let isP ← Meta.isProp cinfo.type
              return (type.pretty, valPp.pretty, validDeps, isP)
          ).run' coreCtx coreSt |>.toIO (fun _ => IO.userError "Meta error")

          -- CHECK IF ABBREV: Look at the ReducibilityHints
          let isAbbrev := match cinfo.hints with
            | ReducibilityHints.abbrev => true
            | _ => false

          decls := decls.concat {
            name := nameStr,
            type := typeStr,
            proofTerm := valStr,
            dependencies := deps,
            isProp := isP,
            isAbbrev := isAbbrev
          }
        catch _ => pure ()

  let json := Json.mkObj [
    ("success", Json.bool true),
    ("module", Json.str env.mainModule.toString),
    ("declarations", toJson decls)
  ]

  IO.println json.pretty
  return 0
""")




# Create a minimal lakefile just to build the extractor tool
with open("lakefile.lean", "w") as f:
    f.write("""
import Lake
open Lake DSL

package extractor_tool

lean_exe extract where
  root := `Extract
  supportInterpreter := true
""")

print("Building updated extractor executable...")
res = subprocess.run(["lake", "build", "extract"], capture_output=True, text=True)
if res.returncode == 0:
    print("✅ Extractor tool is updated and ready.")
else:
    print("❌ Build failed:")
    print(res.stderr)


Building updated extractor executable...
✅ Extractor tool is updated and ready.


## 2.2 Splitter  

In [ ]:
import json
import os
import subprocess
import shutil
import tempfile
import networkx as nx
from pathlib import Path

def run_extractor_on_file(input_path):
    # remove content if not in collab
    exe_paths =["/content/.lake/build/bin/extract", "/content/build/bin/extract"]
    extract_exe = next((p for p in exe_paths if os.path.exists(p)), None)

    if not extract_exe:
        raise FileNotFoundError("Could not find 'extract' executable.")

    with tempfile.TemporaryDirectory() as temp_dir:
        temp_path = Path(temp_dir)
        input_filename = Path(input_path).name
        module_name = Path(input_filename).stem.capitalize()

        shutil.copy(input_path, temp_path / f"{module_name}.lean")

        lakefile_content = f"import Lake\nopen Lake DSL\npackage temp_project\n@[default_target]\nlean_lib {module_name}\n"
        (temp_path / "lakefile.lean").write_text(lakefile_content)

        result = subprocess.run(["lake", "env", extract_exe, f"{module_name}.lean"],
            cwd=temp_dir, capture_output=True, text=True, check=True
        )
        return json.loads(result.stdout)


def split_lean_project(input_file, main_goal_name, output_dir):
    print(f"--- Starting Lean Project Split ---")
    data = run_extractor_on_file(input_file)

    # ==========================================
    # GRAPH WITH PRUNING PHASE
    # ==========================================
    G = nx.DiGraph()
    extracted_names = {decl['name'] for decl in data['declarations']}

    # 1. Build the graph
    for decl in data['declarations']:
        name = decl['name']
        G.add_node(name)
        for dep in decl['dependencies']:
            # Direction: Dependency -> Node that uses it
            G.add_edge(dep, name)

    if main_goal_name not in extracted_names:
        print(f"❌ Error: Main goal '{main_goal_name}' not found.")
        return

    # 2. Find all ancestors of the main goal (everything it depends on, directly or indirectly)
    required_nodes = nx.ancestors(G, main_goal_name)
    required_nodes.add(main_goal_name) # Keep the goal itself!

    # 3. Filter the declarations
    pruned_declarations =[]
    dropped_nodes = []

    for decl in data['declarations']:
        if decl['name'] in required_nodes:
            pruned_declarations.append(decl)
        else:
            dropped_nodes.append(decl['name'])

    if dropped_nodes:
        print(f"✂️  Pruned unused declarations (dead code): {dropped_nodes}")
    # ==========================================

    definitions = {}
    lemmas = {}
    main_goal = None

    # Use the PRUNED list instead of the full data
    for decl in pruned_declarations:
        name = decl['name']
        if name == main_goal_name:
            main_goal = decl
        elif decl.get('isProp', False):
            lemmas[name] = decl
        else:
            definitions[name] = decl

    os.makedirs(output_dir, exist_ok=True)

    # --- Write Defs.lean ---
    if definitions:
        G_defs = nx.DiGraph()
        for name, decl in definitions.items():
            G_defs.add_node(name)
            for dep in decl['dependencies']:
                if dep in definitions:
                    G_defs.add_edge(dep, name)

        sorted_defs = list(nx.topological_sort(G_defs))

        with open(os.path.join(output_dir, 'Defs.lean'), 'w') as f:
            f.write("import Lean\n\n")
            for name in sorted_defs:
                decl = definitions[name]
                kind = "abbrev" if decl.get('isAbbrev', False) else "def"
                f.write(f"{kind} {name} : {decl['type']} := {decl['proofTerm']}\n\n")

    # --- Write individual lemma files ---
    for name, decl in lemmas.items():
        lemma_filename = name.capitalize() + ".lean"
        with open(os.path.join(output_dir, lemma_filename), 'w') as f:
            imports = {"import Lean"}
            for dep in decl['dependencies']:
                if dep in definitions: imports.add("import Defs")
                elif dep in lemmas: imports.add(f"import {dep.capitalize()}")
            f.write("\n".join(sorted(list(imports))) + "\n\n")
            f.write(f"theorem {name} : {decl['type']} := {decl['proofTerm']}\n")

    # --- Write Main.lean ---
    with open(os.path.join(output_dir, 'Main.lean'), 'w') as f:
        imports = {"import Lean"}
        for dep in main_goal['dependencies']:
            if dep in definitions: imports.add("import Defs")
            elif dep in lemmas: imports.add(f"import {dep.capitalize()}")
        f.write("\n".join(sorted(list(imports))) + "\n\n")
        f.write(f"theorem {main_goal['name']} : {main_goal['type']} := {main_goal['proofTerm']}\n")

    # --- Generate lakefile.lean ---
    with open(os.path.join(output_dir, 'lakefile.lean'), 'w') as f:
        f.write("import Lake\nopen Lake DSL\n\npackage split_project\n\n")
        f.write("@[default_target]\nlean_lib Main\n")
        if definitions: f.write("lean_lib Defs\n")
        for name in lemmas: f.write(f"lean_lib {name.capitalize()}\n")

    print(f"✅ Split complete! Files created in: /content/{output_dir}")

# 2.3 Define Monolitic proof

In [ ]:
# 1. Create the monolithic input file
with open("monolith.lean", "w") as f:
    f.write("""
import Lean

abbrev MyNumberType := Nat

def is_special (n : MyNumberType) : Prop := n > 10

-- This one has a real proof
theorem lemma1 (n : MyNumberType) (h : n > 20) : is_special n :=
  by simp [is_special]; exact Nat.le_trans (by decide) h

-- This one is left not as sorry
theorem lemma2 (n : MyNumberType) : n = n :=
  rfl

-- The main goal has a real proof
theorem main_goal (k : MyNumberType) (h : k > 20) : is_special k :=
  lemma1 k h
""")

In [ ]:
# 2. Run the splitter
split_lean_project(
    input_file="monolith.lean",
    main_goal_name="main_goal",
    output_dir="myproj8"
)



--- Starting Lean Project Split ---
✂️  Pruned unused declarations (dead code): ['lemma2']
✅ Split complete! Files created in: /content/myproj8


In [ ]:

# 3. Print the contents of the generated files to verify
print("\n--- 📄 Lemma1.lean (Should have a real proof) ---")
with open("myproj7/Lemma1.lean", "r") as f: print(f.read())

print("--- 📄 Main.lean (Should have a real proof) ---")
with open("myproj7/Main.lean", "r") as f: print(f.read())


--- 📄 Lemma1.lean (Should have a real proof) ---
import Defs
import Lean

theorem lemma1 : ∀ (n : MyNumberType), n > 20 → is_special n := fun n h => Eq.mpr (id gt_iff_lt._simp_1) (Nat.le_trans (of_decide_eq_true (id (Eq.refl true))) h)

--- 📄 Main.lean (Should have a real proof) ---
import Defs
import Lean
import Lemma1

theorem main_goal : ∀ (k : MyNumberType), k > 20 → is_special k := fun k h => lemma1 k h



# Verify

In [ ]:
# 1. Run the command and capture output into a variable
output_lines = !cd myproj8 && lake build
# 2. Convert the list of lines into a single string (like .stdout)
stdout_string = "\n".join(output_lines)
# 3. Print or check the output
print("--- Build Output ---")
print(stdout_string)
#
# 4. Check if it succeeded programmatically
if "Build completed successfully" in stdout_string:
    print("\n✅ Verification Successful!")
else:
    print("\n❌ Verification Failed or Warnings found.")

--- Build Output ---
info: split_project: no previous manifest, creating one from scratch
info: toolchain not updated; no toolchain information found
⣾ [1/1] Running job computation (+ 0 more)
⣷ [2/5] Running Defs (+ 0 more)
⣯ [2/5] Running Defs (+ 0 more)
⣟ [2/5] Running Defs (+ 0 more)
⡿ [2/5] Running Defs (+ 0 more)
⢿ [2/5] Running Defs (+ 0 more)
⣻ [2/5] Running Defs (+ 0 more)
⣽ [2/5] Running Defs (+ 0 more)
⣾ [2/5] Running Defs (+ 0 more)
⣷ [2/5] Running Defs (+ 0 more)
⣯ [2/5] Running Defs (+ 0 more)
⣟ [2/5] Running Defs (+ 0 more)
⡿ [2/5] Running Defs (+ 0 more)
⢿ [2/5] Running Defs (+ 0 more)
⣻ [2/5] Running Defs (+ 0 more)
⣽ [2/5] Running Defs (+ 0 more)
⣾ [2/5] Running Defs (+ 0 more)
⣷ [2/5] Running Defs (+ 0 more)
⣯ [3/5] Running Lemma1 (+ 0 more)
⣟ [3/5] Running Lemma1 (+ 0 more)
⡿ [3/5] Running Lemma1 (+ 0 more)
⢿ [3/5] Running Lemma1 (+ 0 more)
⣻ [3/5] Running Lemma1 (+ 0 more)
⣽ [3/5] Running Lemma1 (+ 0 more)
⣾ [3/5] Running Lemma1 (+ 0 more)
⣷ [3/5] Running Lemma1 (+